In [35]:
import dalex as dx
import pickle as pkl
import data_prep
import pandas as pd
import plotly.graph_objects as go

In [28]:
rf = pkl.load(open("models/rf1.pkl", 'rb'))

In [29]:
df = pd.read_csv("marketing_campaign.csv", sep=";")

x_train, y_train, x_test, y_test = data_prep.prep_data(df)

In [30]:
exp = dx.Explainer(rf, x_train, y_train, model_type="classification")

Preparation of a new explainer is initiated

  -> data              : 1783 rows 31 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 1783 values
  -> model_class       : sklearn.model_selection._search.RandomizedSearchCV (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x000001C4C1DC9D80> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 6.94e-05, mean = 0.139, max = 0.913
  -> model type        : classification will be used
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.502, mean = 3.95e-17, max = 0.713
  -> model_info        : package sklearn

A new explainer has been created!


In [31]:
exp.model_performance()

,recall,precision,f1,accuracy,auc
RandomizedSearchCV,0.875,0.995413,0.93133,0.982053,0.999572


In [32]:
mp = exp.model_parts()

In [33]:
vip = mp.result
vip.sort_values(by="dropout_loss", ascending=False)
exclude_values = ["_baseline_","_full_model_"]
vip = vip.query('variable not in @exclude_values')
vip

,variable,dropout_loss,label
1,complain,0.000417,RandomizedSearchCV
2,education_Basic,0.000419,RandomizedSearchCV
3,marital_status_Widow,0.000444,RandomizedSearchCV
4,education_Master,0.000479,RandomizedSearchCV
5,acceptedcmp2,0.000491,RandomizedSearchCV
6,education_Graduation,0.000528,RandomizedSearchCV
7,marital_status_Together,0.000531,RandomizedSearchCV
8,kidhome,0.000566,RandomizedSearchCV
9,acceptedcmp4,0.000660,RandomizedSearchCV
10,acceptedcmp1,0.000666,RandomizedSearchCV


In [34]:
mp.plot()

In [21]:
pd_rf = exp.model_profile(variables=["acceptedcmp3","recency"])

Calculating ceteris paribus:   0%|          | 0/2 [00:00<?, ?it/s]d:\Programy\miniconda3\envs\bhl9\lib\site-packages\dalex\predict_explanations\_ceteris_paribus\utils.py:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.   0.01 0.02 ... 0.98 0.99 1.  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_data.loc[:, variable] = np.tile(split_points, data.shape[0])
Calculating ceteris paribus:  50%|█████     | 1/2 [00:01<00:01,  1.15s/it]d:\Programy\miniconda3\envs\bhl9\lib\site-packages\dalex\predict_explanations\_ceteris_paribus\utils.py:104: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.    0.99  1.98 ... 97.02 98.01 99.  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new_data.loc[:, variable] = np.tile(split_points, data.shape[0])
Calculating ceteri

In [25]:
pd_rf = pd_rf.result

In [26]:
pd_rf[pd_rf["_vname_"]=="recency"]

,_vname_,_label_,_x_,_yhat_,_ids_
101,recency,RandomizedSearchCV,0.00,0.254771,0
102,recency,RandomizedSearchCV,0.99,0.250712,0
103,recency,RandomizedSearchCV,1.98,0.250203,0
104,recency,RandomizedSearchCV,2.97,0.244388,0
105,recency,RandomizedSearchCV,3.96,0.241094,0
...,...,...,...,...,...
197,recency,RandomizedSearchCV,95.04,0.113619,0
198,recency,RandomizedSearchCV,96.03,0.114411,0
199,recency,RandomizedSearchCV,97.02,0.114494,0
200,recency,RandomizedSearchCV,98.01,0.114781,0


In [38]:
colors = ['blue' if val < max(pd_rf["_yhat_"])-0.1*max(pd_rf["_yhat_"]) else 'green' for val in pd_rf['_yhat_']]

In [ ]:
fig = go.Figure()

for i in range(len(df) - 1):
    fig.add_trace(go.Scatter(
        x=df['_x_'].iloc[i:i+2],
        y=df['_yhat_'].iloc[i:i+2],
        mode='lines',
        line=dict(color=colors[i]),
        showlegend=False
    ))

fig.update_layout(
    title='Line Plot',
    xaxis=dict(title='Wartość'),
    yaxis=dict(title='Prawdopodobieństwo')
)

In [15]:
pd_rf.plot()